In [14]:
!pip install -U -q transformers==4.44.2 bitsandbytes
!pip install -U -q huggingface_hub
!pip install -q flask flask-cors pyngrok flash_attn
!pip install pdf2image
!apt-get install -y poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.5).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [15]:
!pdftoppm -v
!python --version

pdftoppm version 22.02.0
Copyright 2005-2022 The Poppler Developers - http://poppler.freedesktop.org
Copyright 1996-2011 Glyph & Cog, LLC
Python 3.10.12


# Import library

In [21]:
import numpy as np
import torch
import torchvision.transforms as T
from PIL import Image
from torchvision.transforms.functional import InterpolationMode
from transformers import AutoModel, AutoTokenizer
import requests

# Preprocessing image

In [31]:
# Thư viện xử lý ảnh đầu vào (từ source gốc HF)
IMAGENET_MEAN = (0.485, 0.456, 0.406)
IMAGENET_STD = (0.229, 0.224, 0.225)

def build_transform(input_size):
    MEAN, STD = IMAGENET_MEAN, IMAGENET_STD
    transform = T.Compose([
        T.Lambda(lambda img: img.convert('RGB') if img.mode != 'RGB' else img),
        T.Resize((input_size, input_size), interpolation=InterpolationMode.BICUBIC),
        T.ToTensor(),
        T.Normalize(mean=MEAN, std=STD)
    ])
    return transform

def find_closest_aspect_ratio(aspect_ratio, target_ratios, width, height, image_size):
    best_ratio_diff = float('inf')
    best_ratio = (1, 1)
    area = width * height
    for ratio in target_ratios:
        target_aspect_ratio = ratio[0] / ratio[1]
        ratio_diff = abs(aspect_ratio - target_aspect_ratio)
        if ratio_diff < best_ratio_diff:
            best_ratio_diff = ratio_diff
            best_ratio = ratio
        elif ratio_diff == best_ratio_diff:
            if area > 0.5 * image_size * image_size * ratio[0] * ratio[1]:
                best_ratio = ratio
    return best_ratio

def dynamic_preprocess(image, min_num=1, max_num=12, image_size=448, use_thumbnail=False):
    orig_width, orig_height = image.size
    aspect_ratio = orig_width / orig_height

    # calculate the existing image aspect ratio
    target_ratios = set(
        (i, j) for n in range(min_num, max_num + 1) for i in range(1, n + 1) for j in range(1, n + 1) if
        i * j <= max_num and i * j >= min_num)
    target_ratios = sorted(target_ratios, key=lambda x: x[0] * x[1])

    # find the closest aspect ratio to the target
    target_aspect_ratio = find_closest_aspect_ratio(
        aspect_ratio, target_ratios, orig_width, orig_height, image_size)

    # calculate the target width and height
    target_width = image_size * target_aspect_ratio[0]
    target_height = image_size * target_aspect_ratio[1]
    blocks = target_aspect_ratio[0] * target_aspect_ratio[1]

    # resize the image
    resized_img = image.resize((target_width, target_height))
    processed_images = []
    for i in range(blocks):
        box = (
            (i % (target_width // image_size)) * image_size,
            (i // (target_width // image_size)) * image_size,
            ((i % (target_width // image_size)) + 1) * image_size,
            ((i // (target_width // image_size)) + 1) * image_size
        )
        # split the image
        split_img = resized_img.crop(box)
        processed_images.append(split_img)
    assert len(processed_images) == blocks
    if use_thumbnail and len(processed_images) != 1:
        thumbnail_img = image.resize((image_size, image_size))
        processed_images.append(thumbnail_img)
    return processed_images

def load_image(image_file, input_size=448, max_num=12):
    # image = Image.open(requests.get(image_file, stream=True).raw).convert('RGB')
    image = Image.open(image_file).convert('RGB')
    transform = build_transform(input_size=input_size)
    images = dynamic_preprocess(image, image_size=input_size, use_thumbnail=True, max_num=max_num)
    pixel_values = [transform(image) for image in images]
    pixel_values = torch.stack(pixel_values)
    return pixel_values

# Load model

In [18]:
model_name = "5CD-AI/Vintern-1B-v2"
model = AutoModel.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
    trust_remote_code=True,
).eval().cuda()

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True, use_fast=False)
generation_config = dict(max_new_tokens= 512, do_sample=False, num_beams = 3, repetition_penalty=3.5)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/5.33k [00:00<?, ?B/s]

configuration_internvl_chat.py:   0%|          | 0.00/3.80k [00:00<?, ?B/s]

configuration_intern_vit.py:   0%|          | 0.00/5.55k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/5CD-AI/Vintern-1B-v2:
- configuration_intern_vit.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/5CD-AI/Vintern-1B-v2:
- configuration_internvl_chat.py
- configuration_intern_vit.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

tokenizer_config.json:   0%|          | 0.00/3.02k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/3.38M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/265 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/498 [00:00<?, ?B/s]

In [23]:
test_image = 'lichsu_test.png'

pixel_values = load_image(test_image, max_num=12).to(torch.bfloat16).cuda()
generation_config = dict(max_new_tokens= 1024, do_sample=False, num_beams = 3, repetition_penalty=2.5)

question = '''<image>
Hãy nhận diện và sao chép toàn bộ nội dung văn bản từ hình ảnh này. Đảm bảo sao chép đầy đủ và chính xác từng từ, kể cả các chi tiết nhỏ nhất.
'''

response, history = model.chat(tokenizer, pixel_values, question, generation_config, history=None, return_history=True)
print(f'User: {question}\nAssistant: {response}')

Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


User: <image>
Hãy nhận diện và sao chép toàn bộ nội dung văn bản từ hình ảnh này. Đảm bảo sao chép đầy đủ và chính xác từng từ, kể cả các chi tiết nhỏ nhất.

Assistant: Đoạn văn cảm nhận về lịch sử đất nước - Mẫu 7
Đất nước Việt Nam đã trải qua hàng nghìn năm dựng nước và giữ nước. Nhân dân ta đã phải đối mặt với rất nhiều kẻ thù xâm lược nguy hiểm. Nhưng trong bất cứ hoàn cảnh nào, chúng ta vẫn giữ vững quyết tâm bảo vệ chủ quyền của đất nước. Đó là hình ảnh vị anh hùng làng Gióng trong truyền thuyết Thánh Gióng. Người tráng sĩ ấy đã chiến đấu bằng tất cả tinh thần yêu nước, lòng cảm thủ giặc của nhân dân. Sức mạnh của Gióng không chỉ tượng trưng cho sức mạnh của tinh thần đoàn kết toàn dân, đó còn là sức mạnh của sự kết hợp giữa con người và thiên nhiên, bằng cả vũ khí thô sơ (tre) và hiện đại (roi sắt). Hay vị chủ tướng Lê Lợi trong truyền thuyết Sự tích Hồ Gươm. Một con người tài năng, dũng cảm. Dưới sự lãnh đạo của ông, mọi trận chiến của nghĩa quân Lam Sơn đều bách chiến bách thắ

# Ngrok


In [24]:
# Setup Ngrok Token
from google.colab import userdata
from flask import Flask, jsonify, request
from flask_cors import CORS
from pyngrok import ngrok

authtoken = userdata.get("ngrok_token")
ngrok.set_auth_token(authtoken)

In [ ]:
# Viết code Flask để expose ra API

# Initialize Flask app
app = Flask(__name__)
CORS(app)


prompt = '''<image>
Hãy nhận diện và sao chép toàn bộ nội dung văn bản từ hình ảnh này. Đảm bảo sao chép đầy đủ và chính xác từng từ, kể cả các chi tiết nhỏ nhất.
'''


@app.route('/ocr', methods=['POST'])
def index():
    data = request.json
    image_url = data.get('image_url', None)

    response_message = ocr_by_llm(image_url, prompt)

    return jsonify({
        "response_message": response_message
    })

def ocr_by_llm(image_url, prompt):
    # image = Image.open(requests.get(image_url, stream=True).raw)

    pixel_values = load_image(image_url, max_num=6).to(torch.bfloat16).cuda()

    response_message = model.chat(tokenizer, pixel_values, prompt, generation_config)

    del pixel_values

    print(response_message)
    return response_message

if __name__ == '__main__':

    ngrok_url = ngrok.connect(5555)
    print(ngrok_url)

    app.run(port=5555)

In [57]:
# Viết code Flask để expose ra API
import os
# Initialize Flask app
app = Flask(__name__)
CORS(app)


# prompt = '''<image>
# Hãy nhận diện và sao chép toàn bộ văn bản từ hình ảnh này, bao gồm cả tiêu đề, nội dung và các chi tiết nhỏ nhất. Chỉ trả về chính xác nội dung văn bản được trích xuất từ hình ảnh mà không thêm bất kỳ mô tả hoặc thông tin nào khác.
# '''
# prompt='''<image>
# Hãy trích xuất chính xác toàn bộ nội dung văn bản có trong hình ảnh này, bao gồm cả tiêu đề, các đoạn văn, danh sách, số liệu, và ký tự đặc biệt. Đảm bảo không bỏ sót bất kỳ từ hoặc câu nào, kể cả các chi tiết nhỏ nhất. Chỉ trả về văn bản gốc mà không thêm hoặc sửa đổi bất kỳ thông tin nào.
# '''

promt='''<image>
Hãy ghi lại toàn bộ nội dung văn bản có trong ảnh này, bao gồm mọi chữ viết ở trên, dưới, hoặc xung quanh hình ảnh. Đảm bảo không bỏ sót bất kỳ từ, số liệu hoặc câu nào. Chỉ trả về nội dung văn bản, không thay đổi hay thêm bớt bất kỳ chi tiết nào.'''
# prompt = '''<image>\n Hãy ghi lại chi tiết từng chữ có trong ảnh, chỉ cần ghi thôi không cần phải mô tả gì cả, ĐỪNG thay đổi nội dung và hãy ghi TOÀN bộ nội dung kể cả phần tóm tắt đầu bài.'''

UPLOAD_FOLDER = '/content/images'
if not os.path.exists(UPLOAD_FOLDER):
    os.makedirs(UPLOAD_FOLDER)

@app.route('/upload', methods=['POST'])
def upload_image():
    if 'file' not in request.files:
        return jsonify({"error": "No file provided"}), 400

    file = request.files['file']
    if file.filename == '':
        return jsonify({"error": "No file selected"}), 400

    # Lưu file ảnh vào thư mục
    file_path = os.path.join(UPLOAD_FOLDER, file.filename)
    file.save(file_path)
    return jsonify({"message": "File uploaded successfully", "file_path": file_path})

@app.route('/ocr', methods=['POST'])
def index():
    data = request.json
    image_url = data.get('image_url', None)

    response_message = ocr_by_llm(image_url, prompt)

    return jsonify({
        "response_message": response_message
    })

def ocr_by_llm(image_url, prompt):
    # image = Image.open(requests.get(image_url, stream=True).raw)

    pixel_values = load_image(image_url, max_num=6).to(torch.bfloat16).cuda()

    response_message = model.chat(tokenizer, pixel_values, prompt, generation_config)

    del pixel_values

    print(response_message)
    return response_message

if __name__ == '__main__':

    ngrok_url = ngrok.connect(5555)
    print(ngrok_url)

    app.run(port=5555)

NgrokTunnel: "https://a0a1-34-87-45-51.ngrok-free.app" -> "http://localhost:5555"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5555
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [27/Dec/2024 09:45:08] "POST /upload HTTP/1.1" 200 -
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
INFO:werkzeug:127.0.0.1 - - [27/Dec/2024 09:45:50] "POST /ocr HTTP/1.1" 200 -


CHƯƠNG II
LIÊN XÔ VÀ CÁC NƯỚC ĐÔNG ÂU (1945 - 1991)
LIÊN BANG NGA (1991 - 2000)
Bài 2
LIÊN XÔ VÀ CÁC NƯỚC ĐÔNG ÂU (1945 - 1991)
LIÊN BANG NGA (1991 - 2000)
Sau khi Chiến tranh thế giới thứ hai kết thúc, Liên Xô đã nhanh chóng khôi phục đất nước, thực hiện nhiều kế hoạch 5 năm xây dựng chủ nghĩa xã hội. Các nước Đông Âu được giải phóng đã tiến hành cách mạng dân chủ nhân dân, tiến lên chủ nghĩa xã hội.
Từ cuối những năm 80 của thế kỷ XX, cuộc khủng hoảng chính trị và kinh tế - xã hội ở Liên Xô và các nước Đông Âu đã dẫn đến sự tan rã của chế độ xã hội chủ nghĩa ở các nước này. Nhà nước Liên bang Nga kế thừa địa vị và quyền lợi hợp pháp của Liên Xô.
I - LIÊN XÔ VÀ CÁC NƯỚC ĐÔNG ÂU TỪ NĂM 1945 ĐẾN GIỮA NHỮNG NĂM 70
1. Liên Xô
a) Công cuộc khôi phục kinh tế (1945 - 1950)
Liên Xô là nước chịu tổn thất nặng nề nhất trong Chiến tranh thế giới thứ hai: khoảng 27 triệu người chết, 1 710 thành phố, hơn 7 vạn làng mạc, gần 32 000 xí nghiệp bị tàn phá nặng nề.
Với tinh thần tự lực tự cường, nhân d

INFO:werkzeug:127.0.0.1 - - [27/Dec/2024 09:45:54] "POST /upload HTTP/1.1" 200 -
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
INFO:werkzeug:127.0.0.1 - - [27/Dec/2024 09:46:31] "POST /ocr HTTP/1.1" 200 -


Khoa học - kĩ thuật phát triển nhanh chóng. Năm 1949, Liên Xô đã chế tạo thành công bom nguyên tử, phá thế độc quyền vũ khí nguyên tử của Mỹ.
b) Liên Xô tiếp tục xây dựng chủ nghĩa xã hội (từ năm 1950 đến nửa đầu những năm 70)
Trong thời gian này, Liên Xô đã thực hiện nhiều kế hoạch dài hạn nhằm tiếp tục xây dựng cơ sở vật chất - kĩ thuật của chủ nghĩa xã hội. Các kế hoạch này về cơ bản đều hoàn thành với nhiều thành tựu lớn.
Liên Xô trở thành cường quốc công nghiệp đứng thứ hai trên thế giới (sau Mỹ), một số ngành công nghiệp có sản lượng cao vào loại nhất thế giới như dầu mỏ, than, thép v.v... Liên Xô đi đầu trong công nghiệp vũ trụ, công nghiệp điện hạt nhân.
Tuy gặp nhiều khó khăn, sản xuất nông nghiệp vẫn đạt được nhiều thành tựu. Sản lượng nông phẩm trong những năm 60 tăng trung bình hàng năm là 16%.
Về khoa học - kĩ thuật, năm 1957 Liên Xô là nước đầu tiên phóng thành công vệ tinh nhân tạo. Năm 1961, Liên Xô đã phóng con tàu vũ trụ đưa nhà du hành vũ trụ I. Gagarin bay vòng quan

INFO:werkzeug:127.0.0.1 - - [27/Dec/2024 09:46:36] "POST /upload HTTP/1.1" 200 -
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
INFO:werkzeug:127.0.0.1 - - [27/Dec/2024 09:46:53] "POST /ocr HTTP/1.1" 200 -


Cộng hòa Nhân dân Ba Lan (1944), Cộng hòa Nhân dân Rumani (1944), Cộng hòa Nhân dân Hunggari (1945), Cộng hòa Tiệp Khắc (1945), Liên bang (1946), Cộng hòa Nhân dân Anbani (1946).
